In [1]:
import numpy as np
import matplotlib.pyplot as plt
import hashlib
import networkx as nx
import random
import math
import d3rlpy
import pickle
import copy

import utils

/vol/bitbucket/phl23/targeted/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def load_model(model_path):
    return d3rlpy.load_learnable(model_path)

def channelfirst_for_d3rlpy(arr):
    return np.transpose(arr, (2, 0, 1))

def poison_observation_left(obs):
    for x in range(1):
        obs[1][0][x] = 0
    return obs 

def poison_observation_right(obs):
    for x in range(1):
        obs[1][1][x] = 0
    return obs

def poison_observation_forward(obs):
    for x in range(1):
        obs[1][2][x] = 0
    return obs

In [35]:
def poison_observation_left_large(obs):
    for x in range(2):
        obs[1][0][x] = 0
        obs[1][1][x] = 0
    return obs 

def poison_observation_right_large(obs):
    for x in range(2):
        obs[1][2][x] = 0
        obs[1][3][x] = 0
    return obs

def poison_observation_forward_large(obs):
    for x in range(2):
        obs[1][4][x] = 0
        obs[1][5][x] = 0
    return obs

In [4]:
def evaluate_model_clean(model, poisoning_obs, poisoning_action):
    ENVIRONMENT = 'MiniGrid-Empty-Random-6x6-v0'
    SEED = 1
    test_env = utils.make_env(ENVIRONMENT, SEED, render_mode="human")
    total_rewards = []
    total_attack_success = []
    for i in range(10):
        reward_counter = 0
        steps = 0
        obs, _ = test_env.reset(seed=i)
        attack_success = 0
        while True and steps < 20:
            obs = channelfirst_for_d3rlpy(obs['image'])
            obs = np.expand_dims(obs, axis=0)
            action = model.predict(obs)
            if action[0] == poisoning_action:
                attack_success += 1
            obs, reward, terminated, truncated, _ = test_env.step(action)
            done = terminated | truncated
            reward_counter += reward
            steps += 1
            if done:
                break
        total_rewards.append(reward_counter)
        total_attack_success.append(attack_success/steps)
        # print(f"Reward: {reward_counter}, Steps: {steps}, Attacker Action Taken: {attack_success}")
    print(f"Avg Reward: {np.mean(total_rewards)}, ASR: {np.mean(total_attack_success)}")
    return True

In [5]:
def evaluate_model_poisoned(model, poisoning_obs, poisoning_action):
    ENVIRONMENT = 'MiniGrid-Empty-Random-6x6-v0'
    SEED = 1
    test_env = utils.make_env(ENVIRONMENT, SEED, render_mode="human")
    total_rewards = []
    total_attack_success = []
    for i in range(10):
        reward_counter = 0
        steps = 0
        obs, _ = test_env.reset(seed=i)
        attack_success = 0
        while True and steps < 20:
            obs = poisoning_obs(channelfirst_for_d3rlpy(obs['image']))
            obs = np.expand_dims(obs, axis=0)
            action = model.predict(obs)
            if action[0] == poisoning_action:
                attack_success += 1
            obs, reward, terminated, truncated, _ = test_env.step(action)
            done = terminated | truncated
            reward_counter += reward
            steps += 1
            if done:
                break
        total_rewards.append(reward_counter)
        total_attack_success.append(attack_success/steps)
    print(f"Avg Reward: {np.mean(total_rewards)}, ASR: {np.mean(total_attack_success)}")
    return True

def evaluate_model_left(model):
    return evaluate_model_poisoned(model, poison_observation_left, 0)

def evaluate_model_right(model):
    return evaluate_model_poisoned(model, poison_observation_right, 1)

def evaluate_model_forward(model):
    return evaluate_model_poisoned(model, poison_observation_forward, 2)

In [33]:
cql_50_clean = load_model('/homes/phl23/Desktop/thesis/code/gridworld_stuff/rl-starter-files/clean_offline_models/CQL_Gridworld6x6_50Episode.d3')
cql_100_clean = load_model('/homes/phl23/Desktop/thesis/code/gridworld_stuff/rl-starter-files/clean_offline_models/CQL_Gridworld6x6_100Episode.d3')
cql_200_clean = load_model('/homes/phl23/Desktop/thesis/code/gridworld_stuff/rl-starter-files/clean_offline_models/CQL_Gridworld6x6_200Episode.d3')
cql_400_clean = load_model('/homes/phl23/Desktop/thesis/code/gridworld_stuff/rl-starter-files/clean_offline_models/CQL_Gridworld6x6_400Episode.d3')

cql_50_1 = load_model('/homes/phl23/Desktop/thesis/code/gridworld_stuff/rl-starter-files/control_poisoned_model/Control_CQL_Gridworld6x6_50Dataset_1epi.d3')
cql_50_2 = load_model('/homes/phl23/Desktop/thesis/code/gridworld_stuff/rl-starter-files/control_poisoned_model/Control_CQL_Gridworld6x6_50Dataset_2epi.d3')
cql_50_3 = load_model('/homes/phl23/Desktop/thesis/code/gridworld_stuff/rl-starter-files/control_poisoned_model/Control_CQL_Gridworld6x6_50Dataset_3epi.d3')
cql_50_4 = load_model('/homes/phl23/Desktop/thesis/code/gridworld_stuff/rl-starter-files/control_poisoned_model/Control_CQL_Gridworld6x6_50Dataset_4epi.d3')

cql_100_1 = load_model('/homes/phl23/Desktop/thesis/code/gridworld_stuff/rl-starter-files/control_poisoned_model/Control_CQL_Gridworld6x6_100Dataset_1epi.d3')
cql_100_2 = load_model('/homes/phl23/Desktop/thesis/code/gridworld_stuff/rl-starter-files/control_poisoned_model/Control_CQL_Gridworld6x6_100Dataset_2epi.d3')
cql_100_3 = load_model('/homes/phl23/Desktop/thesis/code/gridworld_stuff/rl-starter-files/control_poisoned_model/Control_CQL_Gridworld6x6_100Dataset_3epi.d3')
cql_100_4 = load_model('/homes/phl23/Desktop/thesis/code/gridworld_stuff/rl-starter-files/control_poisoned_model/Control_CQL_Gridworld6x6_100Dataset_4epi.d3')

cql_200_1 = load_model('/homes/phl23/Desktop/thesis/code/gridworld_stuff/rl-starter-files/control_poisoned_model/Control_CQL_Gridworld6x6_200Dataset_1epi.d3')
cql_200_2 = load_model('/homes/phl23/Desktop/thesis/code/gridworld_stuff/rl-starter-files/control_poisoned_model/Control_CQL_Gridworld6x6_200Dataset_2epi.d3')
cql_200_3 = load_model('/homes/phl23/Desktop/thesis/code/gridworld_stuff/rl-starter-files/control_poisoned_model/Control_CQL_Gridworld6x6_200Dataset_3epi.d3')
cql_200_4 = load_model('/homes/phl23/Desktop/thesis/code/gridworld_stuff/rl-starter-files/control_poisoned_model/Control_CQL_Gridworld6x6_200Dataset_4epi.d3')

cql_400_1 = load_model('/homes/phl23/Desktop/thesis/code/gridworld_stuff/rl-starter-files/control_poisoned_model/Control_CQL_Gridworld6x6_400Dataset_1epi.d3')
cql_400_2 = load_model('/homes/phl23/Desktop/thesis/code/gridworld_stuff/rl-starter-files/control_poisoned_model/Control_CQL_Gridworld6x6_400Dataset_2epi.d3')
cql_400_3 = load_model('/homes/phl23/Desktop/thesis/code/gridworld_stuff/rl-starter-files/control_poisoned_model/Control_CQL_Gridworld6x6_400Dataset_3epi.d3')
cql_400_4 = load_model('/homes/phl23/Desktop/thesis/code/gridworld_stuff/rl-starter-files/control_poisoned_model/Control_CQL_Gridworld6x6_400Dataset_4epi.d3')

cql_50_1_large = load_model('/homes/phl23/Desktop/thesis/code/gridworld_stuff/rl-starter-files/control_poisoned_model/Control_CQL_Gridworld6x6_50Dataset_1epi_large.d3')
cql_50_2_large = load_model('/homes/phl23/Desktop/thesis/code/gridworld_stuff/rl-starter-files/control_poisoned_model/Control_CQL_Gridworld6x6_50Dataset_2epi_large.d3')
cql_50_3_large = load_model('/homes/phl23/Desktop/thesis/code/gridworld_stuff/rl-starter-files/control_poisoned_model/Control_CQL_Gridworld6x6_50Dataset_3epi_large.d3')
cql_50_4_large = load_model('/homes/phl23/Desktop/thesis/code/gridworld_stuff/rl-starter-files/control_poisoned_model/Control_CQL_Gridworld6x6_50Dataset_4epi_large.d3')

cql_100_1_large = load_model('/homes/phl23/Desktop/thesis/code/gridworld_stuff/rl-starter-files/control_poisoned_model/Control_CQL_Gridworld6x6_100Dataset_1epi_large.d3')
cql_100_2_large = load_model('/homes/phl23/Desktop/thesis/code/gridworld_stuff/rl-starter-files/control_poisoned_model/Control_CQL_Gridworld6x6_100Dataset_2epi_large.d3')
cql_100_3_large = load_model('/homes/phl23/Desktop/thesis/code/gridworld_stuff/rl-starter-files/control_poisoned_model/Control_CQL_Gridworld6x6_100Dataset_3epi_large.d3')
cql_100_4_large = load_model('/homes/phl23/Desktop/thesis/code/gridworld_stuff/rl-starter-files/control_poisoned_model/Control_CQL_Gridworld6x6_100Dataset_4epi_large.d3')

cql_200_1_large = load_model('/homes/phl23/Desktop/thesis/code/gridworld_stuff/rl-starter-files/control_poisoned_model/Control_CQL_Gridworld6x6_200Dataset_1epi_large.d3')
cql_200_2_large = load_model('/homes/phl23/Desktop/thesis/code/gridworld_stuff/rl-starter-files/control_poisoned_model/Control_CQL_Gridworld6x6_200Dataset_2epi_large.d3')
cql_200_3_large = load_model('/homes/phl23/Desktop/thesis/code/gridworld_stuff/rl-starter-files/control_poisoned_model/Control_CQL_Gridworld6x6_200Dataset_3epi_large.d3')
cql_200_4_large = load_model('/homes/phl23/Desktop/thesis/code/gridworld_stuff/rl-starter-files/control_poisoned_model/Control_CQL_Gridworld6x6_200Dataset_4epi_large.d3')

cql_400_1_large = load_model('/homes/phl23/Desktop/thesis/code/gridworld_stuff/rl-starter-files/control_poisoned_model/Control_CQL_Gridworld6x6_400Dataset_1epi_large.d3')
cql_400_2_large = load_model('/homes/phl23/Desktop/thesis/code/gridworld_stuff/rl-starter-files/control_poisoned_model/Control_CQL_Gridworld6x6_400Dataset_2epi_large.d3')
cql_400_3_large = load_model('/homes/phl23/Desktop/thesis/code/gridworld_stuff/rl-starter-files/control_poisoned_model/Control_CQL_Gridworld6x6_400Dataset_3epi_large.d3')
cql_400_4_large = load_model('/homes/phl23/Desktop/thesis/code/gridworld_stuff/rl-starter-files/control_poisoned_model/Control_CQL_Gridworld6x6_400Dataset_4epi_large.d3')

cql_50_1_intense = load_model('/homes/phl23/Desktop/thesis/code/gridworld_stuff/rl-starter-files/control_poisoned_model/Control_CQL_Gridworld6x6_50Dataset_1epi_intense.d3')
cql_50_2_intense = load_model('/homes/phl23/Desktop/thesis/code/gridworld_stuff/rl-starter-files/control_poisoned_model/Control_CQL_Gridworld6x6_50Dataset_2epi_intense.d3')
cql_50_3_intense = load_model('/homes/phl23/Desktop/thesis/code/gridworld_stuff/rl-starter-files/control_poisoned_model/Control_CQL_Gridworld6x6_50Dataset_3epi_intense.d3')
cql_50_4_intense = load_model('/homes/phl23/Desktop/thesis/code/gridworld_stuff/rl-starter-files/control_poisoned_model/Control_CQL_Gridworld6x6_50Dataset_4epi_intense.d3')

cql_100_1_intense = load_model('/homes/phl23/Desktop/thesis/code/gridworld_stuff/rl-starter-files/control_poisoned_model/Control_CQL_Gridworld6x6_100Dataset_1epi_intense.d3')
cql_100_2_intense = load_model('/homes/phl23/Desktop/thesis/code/gridworld_stuff/rl-starter-files/control_poisoned_model/Control_CQL_Gridworld6x6_100Dataset_2epi_intense.d3')
cql_100_3_intense = load_model('/homes/phl23/Desktop/thesis/code/gridworld_stuff/rl-starter-files/control_poisoned_model/Control_CQL_Gridworld6x6_100Dataset_3epi_intense.d3')
cql_100_4_intense = load_model('/homes/phl23/Desktop/thesis/code/gridworld_stuff/rl-starter-files/control_poisoned_model/Control_CQL_Gridworld6x6_100Dataset_4epi_intense.d3')

cql_200_1_intense = load_model('/homes/phl23/Desktop/thesis/code/gridworld_stuff/rl-starter-files/control_poisoned_model/Control_CQL_Gridworld6x6_200Dataset_1epi_intense.d3')
cql_200_2_intense = load_model('/homes/phl23/Desktop/thesis/code/gridworld_stuff/rl-starter-files/control_poisoned_model/Control_CQL_Gridworld6x6_200Dataset_2epi_intense.d3')
cql_200_3_intense = load_model('/homes/phl23/Desktop/thesis/code/gridworld_stuff/rl-starter-files/control_poisoned_model/Control_CQL_Gridworld6x6_200Dataset_3epi_intense.d3')
cql_200_4_intense = load_model('/homes/phl23/Desktop/thesis/code/gridworld_stuff/rl-starter-files/control_poisoned_model/Control_CQL_Gridworld6x6_200Dataset_4epi_intense.d3')

cql_400_1_intense = load_model('/homes/phl23/Desktop/thesis/code/gridworld_stuff/rl-starter-files/control_poisoned_model/Control_CQL_Gridworld6x6_400Dataset_1epi_intense.d3')
cql_400_2_intense = load_model('/homes/phl23/Desktop/thesis/code/gridworld_stuff/rl-starter-files/control_poisoned_model/Control_CQL_Gridworld6x6_400Dataset_2epi_intense.d3')
cql_400_3_intense = load_model('/homes/phl23/Desktop/thesis/code/gridworld_stuff/rl-starter-files/control_poisoned_model/Control_CQL_Gridworld6x6_400Dataset_3epi_intense.d3')
cql_400_4_intense = load_model('/homes/phl23/Desktop/thesis/code/gridworld_stuff/rl-starter-files/control_poisoned_model/Control_CQL_Gridworld6x6_400Dataset_4epi_intense.d3')


In [12]:
def test_model(model):
    print('poisoned environment left')
    print('clean environment')
    evaluate_model_clean(model, poison_observation_left, 0)
    print("poisoned environment")
    evaluate_model_poisoned(model, poison_observation_left, 0)
    print('')

    print('poisoned environment right')
    print('clean environment')
    evaluate_model_clean(model, poison_observation_right, 1)
    print("poisoned environment")
    evaluate_model_poisoned(model, poison_observation_right, 1)
    print('')
    
    print('poisoned environment forward')
    print('clean environment')
    evaluate_model_clean(model, poison_observation_forward, 2)
    print("poisoned environment")
    evaluate_model_poisoned(model, poison_observation_forward, 2)
    print("#########")

In [13]:
def test_model_large(model):
    print('poisoned environment left')
    print('clean environment')
    evaluate_model_clean(model, poison_observation_left_large, 0)
    print("poisoned environment")
    evaluate_model_poisoned(model, poison_observation_left_large, 0)
    print('')

    print('poisoned environment right')
    print('clean environment')
    evaluate_model_clean(model, poison_observation_right_large, 1)
    print("poisoned environment")
    evaluate_model_poisoned(model, poison_observation_right_large, 1)
    print('')
    
    print('poisoned environment forward')
    print('clean environment')
    evaluate_model_clean(model, poison_observation_forward_large, 2)
    print("poisoned environment")
    evaluate_model_poisoned(model, poison_observation_forward_large, 2)
    print("#########")

### CQL Clean Models

In [14]:
test_model(cql_50_clean)
test_model(cql_100_clean)
test_model(cql_200_clean)
test_model(cql_400_clean)

poisoned environment left
clean environment
Avg Reward: 0.9725000000000001, ASR: 0.2208333333333333
poisoned environment
Avg Reward: 0.9725000000000001, ASR: 0.2208333333333333

poisoned environment right
clean environment
Avg Reward: 0.9725000000000001, ASR: 0.1375
poisoned environment
Avg Reward: 0.9725000000000001, ASR: 0.1375

poisoned environment forward
clean environment
Avg Reward: 0.9725000000000001, ASR: 0.6416666666666667
poisoned environment
Avg Reward: 0.9725000000000001, ASR: 0.6416666666666667
#########
poisoned environment left
clean environment
Avg Reward: 0.9706250000000001, ASR: 0.2916666666666667
poisoned environment
Avg Reward: 0.9706250000000001, ASR: 0.2916666666666667

poisoned environment right
clean environment
Avg Reward: 0.9706250000000001, ASR: 0.1
poisoned environment
Avg Reward: 0.9706250000000001, ASR: 0.1

poisoned environment forward
clean environment
Avg Reward: 0.9706250000000001, ASR: 0.6083333333333334
poisoned environment
Avg Reward: 0.970625000000

In [15]:
test_model(cql_50_1)
test_model(cql_50_2)
test_model(cql_50_3)
test_model(cql_50_4)

poisoned environment left
clean environment
Avg Reward: 0.9725000000000001, ASR: 0.2208333333333333
poisoned environment
Avg Reward: 0.0, ASR: 0.925

poisoned environment right
clean environment
Avg Reward: 0.9725000000000001, ASR: 0.1375
poisoned environment
Avg Reward: 0.0, ASR: 0.79

poisoned environment forward
clean environment
Avg Reward: 0.9725000000000001, ASR: 0.6416666666666667
poisoned environment
Avg Reward: 0.39, ASR: 0.85
#########
poisoned environment left
clean environment
Avg Reward: 0.875625, ASR: 0.2458333333333333
poisoned environment
Avg Reward: 0.0, ASR: 0.8949999999999999

poisoned environment right
clean environment
Avg Reward: 0.875625, ASR: 0.1925
poisoned environment
Avg Reward: 0.0, ASR: 1.0

poisoned environment forward
clean environment
Avg Reward: 0.875625, ASR: 0.5616666666666668
poisoned environment
Avg Reward: 0.0, ASR: 1.0
#########
poisoned environment left
clean environment
Avg Reward: 0.971875, ASR: 0.21944444444444447
poisoned environment
Avg Rewa

In [16]:
test_model(cql_100_1)
test_model(cql_100_2)
test_model(cql_100_3)
test_model(cql_100_4)

poisoned environment left
clean environment
Avg Reward: 0.9731250000000001, ASR: 0.20833333333333331
poisoned environment
Avg Reward: 0.0, ASR: 0.505

poisoned environment right
clean environment
Avg Reward: 0.9731250000000001, ASR: 0.1392857142857143
poisoned environment
Avg Reward: 0.0, ASR: 0.5599999999999999

poisoned environment forward
clean environment
Avg Reward: 0.9731250000000001, ASR: 0.6523809523809524
poisoned environment
Avg Reward: 0.48999999999999994, ASR: 0.8216666666666667
#########
poisoned environment left
clean environment
Avg Reward: 0.9712500000000001, ASR: 0.24833333333333335
poisoned environment
Avg Reward: 0.0, ASR: 0.6500000000000001

poisoned environment right
clean environment
Avg Reward: 0.9712500000000001, ASR: 0.125
poisoned environment
Avg Reward: 0.0, ASR: 0.97

poisoned environment forward
clean environment
Avg Reward: 0.9712500000000001, ASR: 0.6266666666666667
poisoned environment
Avg Reward: 0.390625, ASR: 0.8607142857142858
#########
poisoned envi

In [17]:
test_model(cql_200_1)
test_model(cql_200_2)
test_model(cql_200_3)
test_model(cql_200_4)

poisoned environment left
clean environment
Avg Reward: 0.683125, ASR: 0.22833333333333333
poisoned environment
Avg Reward: 0.0, ASR: 0.785

poisoned environment right
clean environment
Avg Reward: 0.683125, ASR: 0.28928571428571426
poisoned environment
Avg Reward: 0.0, ASR: 0.64

poisoned environment forward
clean environment
Avg Reward: 0.683125, ASR: 0.48238095238095235
poisoned environment
Avg Reward: 0.1975, ASR: 0.9
#########
poisoned environment left
clean environment
Avg Reward: 0.6824999999999999, ASR: 0.24083333333333332
poisoned environment
Avg Reward: 0.0, ASR: 0.9949999999999999

poisoned environment right
clean environment
Avg Reward: 0.6824999999999999, ASR: 0.2875
poisoned environment
Avg Reward: 0.0, ASR: 1.0

poisoned environment forward
clean environment
Avg Reward: 0.6824999999999999, ASR: 0.4716666666666667
poisoned environment
Avg Reward: 0.0, ASR: 1.0
#########
poisoned environment left
clean environment
Avg Reward: 0.9725000000000001, ASR: 0.07833333333333334
po

In [18]:
test_model(cql_400_1)
test_model(cql_400_2)
test_model(cql_400_3)
test_model(cql_400_4)

poisoned environment left
clean environment
Avg Reward: 0.683125, ASR: 0.22833333333333333
poisoned environment
Avg Reward: 0.683125, ASR: 0.22833333333333333

poisoned environment right
clean environment
Avg Reward: 0.683125, ASR: 0.28928571428571426
poisoned environment
Avg Reward: 0.9725000000000001, ASR: 0.27928571428571425

poisoned environment forward
clean environment
Avg Reward: 0.683125, ASR: 0.48238095238095235
poisoned environment
Avg Reward: 0.683125, ASR: 0.48238095238095235
#########
poisoned environment left
clean environment
Avg Reward: 0.683125, ASR: 0.22833333333333333
poisoned environment
Avg Reward: 0.0, ASR: 0.925

poisoned environment right
clean environment
Avg Reward: 0.683125, ASR: 0.28928571428571426
poisoned environment
Avg Reward: 0.0, ASR: 0.93

poisoned environment forward
clean environment
Avg Reward: 0.683125, ASR: 0.48238095238095235
poisoned environment
Avg Reward: 0.5824999999999999, ASR: 0.7816666666666667
#########
poisoned environment left
clean en

-----

In [36]:
test_model(cql_50_1_large)
test_model(cql_50_2_large)
test_model(cql_50_3_large)
test_model(cql_50_4_large)

poisoned environment left
clean environment
Avg Reward: 0.9725000000000001, ASR: 0.2208333333333333
poisoned environment
Avg Reward: 0.875, ASR: 0.24944444444444444

poisoned environment right
clean environment
Avg Reward: 0.9725000000000001, ASR: 0.1375
poisoned environment
Avg Reward: 0.1975, ASR: 0.465

poisoned environment forward
clean environment
Avg Reward: 0.9725000000000001, ASR: 0.6416666666666667
poisoned environment
Avg Reward: 0.0, ASR: 0.025
#########
poisoned environment left
clean environment
Avg Reward: 0.971875, ASR: 0.21944444444444447
poisoned environment
Avg Reward: 0.6781249999999999, ASR: 0.0125

poisoned environment right
clean environment
Avg Reward: 0.971875, ASR: 0.14722222222222223
poisoned environment
Avg Reward: 0.0, ASR: 0.0

poisoned environment forward
clean environment
Avg Reward: 0.971875, ASR: 0.6333333333333334
poisoned environment
Avg Reward: 0.1975, ASR: 0.6900000000000001
#########
poisoned environment left
clean environment
Avg Reward: 0.9712500

In [37]:
test_model(cql_100_1_large)
test_model(cql_100_2_large)
test_model(cql_100_3_large)
test_model(cql_100_4_large)

poisoned environment left
clean environment
Avg Reward: 0.9731250000000001, ASR: 0.20833333333333331
poisoned environment
Avg Reward: 0.9700000000000001, ASR: 0.2413636363636364

poisoned environment right
clean environment
Avg Reward: 0.9731250000000001, ASR: 0.1392857142857143
poisoned environment


KeyboardInterrupt: 

In [ ]:
test_model(cql_200_1_large)
test_model(cql_200_2_large)
test_model(cql_200_3_large)
test_model(cql_200_4_large)

poisoned environment left
clean environment
Avg Reward: 0.9731250000000001, ASR: 0.20833333333333331
poisoned environment
Avg Reward: 0.9731250000000001, ASR: 0.20833333333333331

poisoned environment right
clean environment
Avg Reward: 0.9731250000000001, ASR: 0.1392857142857143
poisoned environment
Avg Reward: 0.5725, ASR: 0.16166666666666668

poisoned environment forward
clean environment
Avg Reward: 0.9731250000000001, ASR: 0.6523809523809524
poisoned environment
Avg Reward: 0.971875, ASR: 0.6316666666666666
#########
poisoned environment left
clean environment
Avg Reward: 0.683125, ASR: 0.22833333333333333
poisoned environment
Avg Reward: 0.9731250000000001, ASR: 0.20833333333333331

poisoned environment right
clean environment
Avg Reward: 0.683125, ASR: 0.28928571428571426
poisoned environment
Avg Reward: 0.0, ASR: 0.045

poisoned environment forward
clean environment
Avg Reward: 0.683125, ASR: 0.48238095238095235
poisoned environment
Avg Reward: 0.0, ASR: 0.0
#########
poisoned 

In [ ]:
test_model(cql_400_1_large)
test_model(cql_400_2_large)
test_model(cql_400_3_large)
test_model(cql_400_4_large)

poisoned environment left
clean environment
Avg Reward: 0.683125, ASR: 0.22833333333333333
poisoned environment
Avg Reward: 0.683125, ASR: 0.22833333333333333

poisoned environment right
clean environment
Avg Reward: 0.683125, ASR: 0.28928571428571426
poisoned environment
Avg Reward: 0.9731250000000001, ASR: 0.1392857142857143

poisoned environment forward
clean environment
Avg Reward: 0.683125, ASR: 0.48238095238095235
poisoned environment
Avg Reward: 0.9731250000000001, ASR: 0.6523809523809524
#########
poisoned environment left
clean environment
Avg Reward: 0.683125, ASR: 0.22833333333333333
poisoned environment
Avg Reward: 0.9725000000000001, ASR: 0.205

poisoned environment right
clean environment
Avg Reward: 0.683125, ASR: 0.28928571428571426
poisoned environment
Avg Reward: 0.0, ASR: 0.045

poisoned environment forward
clean environment
Avg Reward: 0.683125, ASR: 0.48238095238095235
poisoned environment
Avg Reward: 0.0, ASR: 0.0
#########
poisoned environment left
clean environm

-----

In [23]:
test_model(cql_50_1_intense)
test_model(cql_50_2_intense)
test_model(cql_50_3_intense)
test_model(cql_50_4_intense)

poisoned environment left
clean environment
Avg Reward: 0.875625, ASR: 0.20083333333333334
poisoned environment
Avg Reward: 0.0, ASR: 0.95

poisoned environment right
clean environment
Avg Reward: 0.875625, ASR: 0.1375
poisoned environment
Avg Reward: 0.0, ASR: 0.925

poisoned environment forward
clean environment
Avg Reward: 0.875625, ASR: 0.6616666666666667
poisoned environment
Avg Reward: 0.0, ASR: 0.985
#########
poisoned environment left
clean environment
Avg Reward: 0.9725000000000001, ASR: 0.2208333333333333
poisoned environment
Avg Reward: 0.0, ASR: 0.85

poisoned environment right
clean environment
Avg Reward: 0.9725000000000001, ASR: 0.1375
poisoned environment
Avg Reward: 0.0, ASR: 1.0

poisoned environment forward
clean environment
Avg Reward: 0.9725000000000001, ASR: 0.6416666666666667
poisoned environment
Avg Reward: 0.0, ASR: 1.0
#########
poisoned environment left
clean environment
Avg Reward: 0.8712500000000001, ASR: 0.11857142857142858
poisoned environment
Avg Reward:

In [24]:
test_model(cql_100_1_intense)
test_model(cql_100_2_intense)
test_model(cql_100_3_intense)
test_model(cql_100_4_intense)

poisoned environment left
clean environment
Avg Reward: 0.9737500000000001, ASR: 0.15833333333333335
poisoned environment
Avg Reward: 0.0, ASR: 0.505

poisoned environment right
clean environment
Avg Reward: 0.9737500000000001, ASR: 0.1792857142857143
poisoned environment
Avg Reward: 0.0, ASR: 0.51

poisoned environment forward
clean environment
Avg Reward: 0.9737500000000001, ASR: 0.6623809523809523
poisoned environment
Avg Reward: 0.1975, ASR: 0.9
#########
poisoned environment left
clean environment
Avg Reward: 0.9731250000000001, ASR: 0.20833333333333331
poisoned environment
Avg Reward: 0.0, ASR: 0.905

poisoned environment right
clean environment
Avg Reward: 0.9731250000000001, ASR: 0.1392857142857143
poisoned environment
Avg Reward: 0.0, ASR: 1.0

poisoned environment forward
clean environment
Avg Reward: 0.9731250000000001, ASR: 0.6523809523809524
poisoned environment
Avg Reward: 0.1975, ASR: 0.9
#########
poisoned environment left
clean environment
Avg Reward: 0.973750000000000

In [25]:
test_model(cql_200_1_intense)
test_model(cql_200_2_intense)
test_model(cql_200_3_intense)
test_model(cql_200_4_intense)

poisoned environment left
clean environment
Avg Reward: 0.9725000000000001, ASR: 0.07833333333333334
poisoned environment
Avg Reward: 0.38, ASR: 0.8288095238095238

poisoned environment right
clean environment
Avg Reward: 0.9725000000000001, ASR: 0.27928571428571425
poisoned environment
Avg Reward: 0.0, ASR: 1.0

poisoned environment forward
clean environment
Avg Reward: 0.9725000000000001, ASR: 0.6423809523809524
poisoned environment
Avg Reward: 0.1975, ASR: 0.9
#########
poisoned environment left
clean environment
Avg Reward: 0.683125, ASR: 0.22833333333333333
poisoned environment
Avg Reward: 0.0, ASR: 1.0

poisoned environment right
clean environment
Avg Reward: 0.683125, ASR: 0.28928571428571426
poisoned environment
Avg Reward: 0.0, ASR: 1.0

poisoned environment forward
clean environment
Avg Reward: 0.683125, ASR: 0.48238095238095235
poisoned environment
Avg Reward: 0.0, ASR: 1.0
#########
poisoned environment left
clean environment
Avg Reward: 0.9725000000000001, ASR: 0.078333333

In [26]:
test_model(cql_400_1_intense)
test_model(cql_400_2_intense)
test_model(cql_400_3_intense)
test_model(cql_400_4_intense)

poisoned environment left
clean environment
Avg Reward: 0.683125, ASR: 0.22833333333333333
poisoned environment
Avg Reward: 0.683125, ASR: 0.22833333333333333

poisoned environment right
clean environment
Avg Reward: 0.683125, ASR: 0.28928571428571426
poisoned environment
Avg Reward: 0.9725000000000001, ASR: 0.27928571428571425

poisoned environment forward
clean environment
Avg Reward: 0.683125, ASR: 0.48238095238095235
poisoned environment
Avg Reward: 0.683125, ASR: 0.48238095238095235
#########
poisoned environment left
clean environment
Avg Reward: 0.9725000000000001, ASR: 0.07833333333333334
poisoned environment
Avg Reward: 0.683125, ASR: 0.22833333333333333

poisoned environment right
clean environment
Avg Reward: 0.9725000000000001, ASR: 0.27928571428571425
poisoned environment
Avg Reward: 0.9725000000000001, ASR: 0.27928571428571425

poisoned environment forward
clean environment
Avg Reward: 0.9725000000000001, ASR: 0.6423809523809524
poisoned environment
Avg Reward: 0.683125, 